In [16]:
import os
import logging
import json
import h2o
import numpy as np
import pandas as pd

In [17]:
AZUREML_MODEL_DIR = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/ci-lucazav/code/Users/lucazavarella/mojo-endpoints/online-endpoint/model"
model_path = os.path.join(AZUREML_MODEL_DIR, "MOJO_model10032025.zip")

In [18]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,36 mins 23 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,"1 year, 4 months and 1 day"
H2O_cluster_name:,H2O_from_python_azureuser_t0m2em
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.409 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [19]:
model = h2o.import_mojo(model_path)

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


In [20]:
data = pd.read_excel("data/testing_data_from_knime.xlsx")
data.head()

/anaconda/envs/mojo_env/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Thickness,Lunghezza,Larghezza,Alt_Testata,Altezza fiancata,Altezza svaso,TIPOLOGIA SCATOLA,SUPPORTO,BCT MISURATO,C_Ond1,C_Ond2,CodCarta1,CodCarta2,CodCarta3,CodCarta4,CodCarta5
0,4.85,500,300,67,68,49,ISR,0,1700,1.25,1.36,5,9,4,9,7
1,4.85,400,300,96,65,51,PLST,1,812,1.25,1.36,5,6,4,9,7
2,7.50,600,400,95,76,66,PLST,1,835,1.36,1.46,7,6,7,9,7
3,7.50,500,300,96,70,60,PLST,1,837,1.36,1.46,7,6,7,6,7
4,7.50,500,300,96,70,60,PLST,1,875,1.36,1.46,7,6,7,6,7


In [21]:
X = data.drop('BCT MISURATO', axis=1)
y = data['BCT MISURATO']

In [22]:
X = h2o.H2OFrame(X)
y = h2o.H2OFrame(y.to_frame())

val_pred = model.predict(X)
val_pred

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%


predict
1497.13
1090.68
907.023
853.485
853.485
1497.13
984.571
1064.71
1041.98
1064.71


In [23]:
abs_residuals = (y - val_pred["predict"]).abs()

In [24]:
residuals_np = abs_residuals.as_data_frame().values.flatten()  # shape (N,)
alpha = 0.1  # for a 90% prediction interval
q = np.quantile(residuals_np, 1 - alpha, method="closest_observation")
print(f"Quantile of residuals for alpha={alpha}: {q:.4f}")

Quantile of residuals for alpha=0.1: 170.4836


/anaconda/envs/mojo_env/lib/python3.11/site-packages/h2o/frame.py:1979: H2ODependencyWarning: converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above).
  warnings.warn("converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
